## Deep Learning - Lab 04

* Full name: Đinh Anh Huy
* Student ID: 18110103

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from tensorflow.keras.layers import Input, Reshape, Dense, Convolution2D, MaxPool2D, BatchNormalization, ReLU, GlobalAveragePooling2D

In [ ]:
class ResidualBlock(keras.layers.Layer):
  def __init__(self, filters_1=64, filters_2=256, kernel_regularizer=None, down_sampling=False):
    super(ResidualBlock, self).__init__()
    self.filters_1 = filters_1
    self.filters_2 = filters_2
    self.down_sampling = down_sampling
    self.kernel_regularizer = kernel_regularizer

  def get_config(self):
    config = super(ResidualBlock, self).get_config()
    config.update({
        'n_filters': self.n_filters,
        'down_sampling': self.down_sampling,
    })
    return config

  def build(self, input_shape):
    self.projection_shorcut = (int(input_shape[-1]) != self.filters_2) or self.down_sampling
    first_strides = [1,1]
    if self.down_sampling:
      first_strides = [2,2]

    # Convolutional Layer 1
    self.main_conv_1 = Convolution2D(filters=self.filters_1,
                                     kernel_size=[1,1],
                                     strides=first_strides,
                                     padding='same',
                                     kernel_regularizer=self.kernel_regularizer,
                                     activation=None)
    self.main_batch_1 = BatchNormalization()
    self.main_relu_1 = ReLU()

    # Convolutional Layer 2
    self.main_conv_2 = Convolution2D(filters=self.filters_1,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding='same',
                                     kernel_regularizer=self.kernel_regularizer,
                                     activation=None)
    self.main_batch_2 = BatchNormalization()
    self.main_relu_2 = ReLU()

    # Convolutional Layer 3
    self.main_conv_3 = Convolution2D(filters=self.filters_2,
                                     kernel_size=[1,1],
                                     strides=[1,1],
                                     padding='same',
                                     kernel_regularizer=self.kernel_regularizer,
                                     activation=None)
    self.main_batch_3 = BatchNormalization()

    ## Shortcut connection
    if self.projection_shorcut:
      self.shortcut_conv = Convolution2D(filters=self.filters_2,
                                         kernel_size=[1,1],
                                         strides=first_strides,
                                         padding='same',
                                         kernel_regularizer=self.kernel_regularizer,
                                         activation=None)
      self.shortcut_batch = BatchNormalization()
    
    self.main_relu_3 = ReLU()

  def call(self, inputs):
    # Convolutional Layer 1
    main_conv_1 = self.main_conv_1(inputs)
    main_batch_1 = self.main_batch_1(main_conv_1)
    main_relu_1 = self.main_relu_1(main_batch_1)

    # Convolutional Layer 2
    main_conv_2 = self.main_conv_2(main_relu_1)
    main_batch_2 = self.main_batch_2(main_conv_2)
    main_relu_2 = self.main_relu_2(main_batch_2)

    # Convolutional Layer 3
    main_conv_3 = self.main_conv_3(main_relu_2)
    main_batch_3 = self.main_batch_3(main_conv_3)

    if self.projection_shorcut:
      shortcut_conv = self.shortcut_conv(inputs)
      shortcut = self.shortcut_batch(shortcut_conv)
    else:
      shortcut = inputs

    main_add = main_batch_3 + shortcut
    main_relu_3 = self.main_relu_3(main_add)

    return main_relu_3

In [ ]:
l2_regularizer_rate = 1e-4

# Input Layer
inputs = Input(shape=(28,28,1))
# inputs_reshape = Reshape(target_shape=(28,28,1))(inputs)

# Block 1
conv_1 = Convolution2D(filters=64,
                       kernel_size=[7,7],
                       strides=[2,2],
                       padding='same',
                       kernel_regularizer=l2(l2_regularizer_rate),
                       activation=None)(inputs)
batch_1 = BatchNormalization()(conv_1)
relu_1 = ReLU()(batch_1)
maxpool_1 = MaxPool2D(pool_size=[3,3],
                      strides=[2,2])(relu_1)

# Block 2
res2_1 = ResidualBlock(filters_1=64, 
                       filters_2=256,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(maxpool_1)
res2_2 = ResidualBlock(filters_1=64, 
                       filters_2=256,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res2_1)
res2_3 = ResidualBlock(filters_1=64, 
                       filters_2=256,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res2_2)

# Block 3
res3_1 = ResidualBlock(filters_1=128, 
                       filters_2=512,
                       down_sampling=True,
                       kernel_regularizer=l2(l2_regularizer_rate))(res2_3)
res3_2 = ResidualBlock(filters_1=128, 
                       filters_2=512,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res3_1)
res3_3 = ResidualBlock(filters_1=128, 
                       filters_2=512,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res3_2)
res3_4 = ResidualBlock(filters_1=128, 
                       filters_2=512,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res3_3)

# Block 4
res4_1 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=True,
                       kernel_regularizer=l2(l2_regularizer_rate))(res3_4)
res4_2 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_1)
res4_3 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_2)
res4_4 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_3)
res4_5 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_4)
res4_6 = ResidualBlock(filters_1=256, 
                       filters_2=1024,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_5)

# Block 5
res5_1 = ResidualBlock(filters_1=512, 
                       filters_2=2048,
                       down_sampling=True,
                       kernel_regularizer=l2(l2_regularizer_rate))(res4_6)
res5_2 = ResidualBlock(filters_1=512, 
                       filters_2=2048,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res5_1)
res5_3 = ResidualBlock(filters_1=512, 
                       filters_2=2048,
                       down_sampling=False,
                       kernel_regularizer=l2(l2_regularizer_rate))(res5_2)

# Block Output
average_pool = GlobalAveragePooling2D()(res5_3)
softmax = Dense(units=10, activation='softmax')(average_pool)

model = keras.models.Model(inputs=inputs, outputs=softmax)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        3200      
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 6, 64)          0         
_________________________________________________________________
residual_block (ResidualBloc (None, 6, 6, 256)         76928     
_________________________________________________________________
residual_block_1 (ResidualBl (None, 6, 6, 256)         71552 

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train, X_test = X_train/255.0, X_test/255.0

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

print("> Shape of training set  : ", X_train.shape)
print("> Shape of validation set: ", X_valid.shape)
print("> Shape of testing set   : ", X_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
> Shape of training set  :  (54000, 28, 28)
> Shape of validation set:  (6000, 28, 28)
> Shape of testing set   :  (10000, 28, 28)


In [ ]:
# init Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-4/50)

# compile model
model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

# fine-tune the model
history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=35,
                    steps_per_epoch=X_train.shape[0]//128,
                    validation_data=(X_valid, y_valid))

Epoch 1/35
421/421 [==============================] - 48s 57ms/step - loss: 2.9046 - accuracy: 0.6203 - val_loss: 3.6639 - val_accuracy: 0.3440
Epoch 2/35
421/421 [==============================] - 22s 53ms/step - loss: 2.0681 - accuracy: 0.8731 - val_loss: 2.0091 - val_accuracy: 0.8910
Epoch 3/35
421/421 [==============================] - 22s 53ms/step - loss: 1.8655 - accuracy: 0.9351 - val_loss: 1.9399 - val_accuracy: 0.9142
Epoch 4/35
421/421 [==============================] - 22s 53ms/step - loss: 1.7740 - accuracy: 0.9608 - val_loss: 1.8919 - val_accuracy: 0.9333
Epoch 5/35
421/421 [==============================] - 22s 53ms/step - loss: 1.7209 - accuracy: 0.9751 - val_loss: 1.9082 - val_accuracy: 0.9300
Epoch 6/35
421/421 [==============================] - 22s 53ms/step - loss: 1.7006 - accuracy: 0.9778 - val_loss: 1.8683 - val_accuracy: 0.9372
Epoch 7/35
421/421 [==============================] - 22s 52ms/step - loss: 1.6778 - accuracy: 0.9817 - val_loss: 1.8284 - val_accuracy:

In [ ]:
evaluation = model.evaluate(X_test, y_test)
print("Accuracy of model on MNIST dataset: {}, and loss: {}.".format(evaluation[1], evaluation[0]))

313/313 [==============================] - 5s 16ms/step - loss: 0.4599 - accuracy: 0.9881
Accuracy of model on MNIST dataset: 0.988099992275238, and loss: 0.4598911702632904.
